### Loading an image file

imageio module을 통해서 PNG 파일을 불러와 보자.
이미지는 width, height 그리고 (r,g,b) 채널로 이루어진 3차원 Numpy arraylike object이다.

In [1]:
import imageio

In [8]:
!wget https://raw.githubusercontent.com/deep-learning-with-pytorch/dlwpt-code/master/data/p1ch4/image-dog/bobby.jpg

--2021-03-30 06:18:53--  https://raw.githubusercontent.com/deep-learning-with-pytorch/dlwpt-code/master/data/p1ch4/image-dog/bobby.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75801 (74K) [image/jpeg]
Saving to: ‘bobby.jpg’

bobby.jpg           100%[===================>]  74.02K  --.-KB/s    in 0.01s   

2021-03-30 06:18:53 (5.32 MB/s) - ‘bobby.jpg’ saved [75801/75801]



In [9]:
img_arr = imageio.imread('bobby.jpg')
img_arr.shape

(720, 1280, 3)

### Changing the layout



In [12]:
import torch
img = torch.from_numpy(img_arr)
out = img.permute(2,0,1)

위의 코드를 통해서 H\*W\*C 채널은 C\*H\*W로 변경되었다.

이때 주의할 점은 img를 수정한다면 out도 같이 변화하게 된다는 것이다.

batch에 이미지를 N\*C\*H\*W로 저장을 해보자.

In [13]:
batch_size = 3
batch = torch.zeros(batch_size,3,256,256,dtype=torch.uint8)

In [ ]:
!wget -P ./cat/ https://raw.githubusercontent.com/deep-learning-with-pytorch/dlwpt-code/master/data/p1ch4/image-cats/cat1.png
!wget -P ./cat/ https://raw.githubusercontent.com/deep-learning-with-pytorch/dlwpt-code/master/data/p1ch4/image-cats/cat2.png
!wget -P ./cat/ https://raw.githubusercontent.com/deep-learning-with-pytorch/dlwpt-code/master/data/p1ch4/image-cats/cat3.png

In [21]:
import os
data_dir='./cat'
filenames = [name for name in os.listdir(data_dir) if os.path.splitext(name)[-1]=='.png']
for i, filename in enumerate(filenames):
    img_arr = imageio.imread(os.path.join(data_dir,filename))
    img_t = torch.from_numpy(img_arr)
    img_t = img_t.permute(2,0,1)
    img_t=img_t[:3]
    # Alpha 채널이 있을 수 있기 때문에 앞의 3개의 채널만 받는다.
    batch[i]=img_t

데이터를 0에서 1, -1에서 1로 하는 것은 NN을 잘 학습시키기 위해서 필요하다.

이를 위해서 다음과 같은 방법이 있다.
1. 8-bit 에서의 최대값인 255 나누어주는 방법

In [23]:
batch = batch.float()
batch /= 255.0

2. 평균과 분산을 구해 zero mean, unit standard deviation을 가지도록 한다.

In [27]:
n_channels = batch.shape[1]
for c in range(n_channels):
    mean = torch.mean(batch[:,c])
    std = torch.std(batch[:,c])
    batch[:,c]= (batch[:,c] - mean)/std